<a href="https://colab.research.google.com/github/fydaijian/test_faster-r-cnn/blob/master/RPN_11_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import os
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
!nvidia-smi


Wed Nov  6 12:20:54 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.50       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [16]:
path = "/content/gdrive/My Drive"
os.chdir(path)
os.listdir(path)

['Getting started.pdf',
 'Colab Notebooks',
 '无标题文件夹',
 'VOC2007',
 'Getting started.gdoc']

In [0]:
import random
folderpath =  "VOC2007"
trainval_percent = 0.66
train_percent = 0.5
xmlfilepath = 'VOC2007/Annotations'
txtsavepath = 'VOC2007/ImageSets/Main'
total_xml = os.listdir(xmlfilepath)
num = len(total_xml)
list = range(num)
tv = int(num*trainval_percent)
tr = int(tv*train_percent)
trainval = random.sample(list,tv)
train = random.sample(trainval,tr)
ftrainval = open(folderpath + '/ImageSets/Main/trainval.txt', 'w')
ftest = open(folderpath + '/ImageSets/Main/test.txt', 'w')
ftrain = open(folderpath + '/ImageSets/Main/train.txt', 'w')
fval = open(folderpath + '/ImageSets/Main/val.txt', 'w')
for i in list:
    name = total_xml[i][:-4]+'\n'
    if i in trainval:
        ftrainval.write(name)
        if i in train:
            ftrain.write(name)
        else:
            fval.write(name)
    else:
        ftest.write(name)
ftrainval.close()
ftrain.close()
fval.close()
ftest .close()


In [27]:
import os
import cv2
import xml.etree.ElementTree as ET
from tqdm import tqdm
import pickle

def get_data(input_path):
    all_imgs = []
    classes_count = {}
    class_mapping = {}

    # parsing 정보 확인 Flag
    visualise = False

    # pascal voc directory + 2012
    data_paths = [os.path.join(input_path, 'VOC2007')]

    print('Parsing annotation files')
    for data_path in data_paths:

        annot_path = os.path.join(data_path, 'Annotations')
        imgs_path = os.path.join(data_path, 'JPEGImages')

        #ImageSets/Main directory의 4개 파일(train, val, trainval, test)
        imgsets_path_trainval = os.path.join(data_path, 'ImageSets', 'Main', 'trainval.txt')
        imgsets_path_train = os.path.join(data_path, 'ImageSets', 'Main', 'train.txt')
        imgsets_path_val = os.path.join(data_path, 'ImageSets', 'Main', 'val.txt')
        imgsets_path_test = os.path.join(data_path, 'ImageSets', 'Main', 'test.txt')

        trainval_files = []
        train_files = []
        val_files = []
        test_files = []

        with open(imgsets_path_trainval) as f:
            for line in f:
                trainval_files.append(line.strip() + '.jpg')

        with open(imgsets_path_train) as f:
            for line in f:
                train_files.append(line.strip() + '.jpg')

        with open(imgsets_path_val) as f:
            for line in f:
                val_files.append(line.strip() + '.jpg')

        # test-set not included in pascal VOC 2012
        if os.path.isfile(imgsets_path_test):
            with open(imgsets_path_test) as f:
                for line in f:
                    test_files.append(line.strip() + '.jpg')

        # 이미지셋 txt 파일 read 예외처리
        # try:
        #     with open(imgsets_path_trainval) as f:
        #         for line in f:
        #             trainval_files.append(line.strip() + '.jpg')
        # except Exception as e:
        #     print(e)
        #
        # try:
        #     with open(imgsets_path_test) as f:
        #         for line in f:
        #             test_files.append(line.strip() + '.jpg')
        # except Exception as e:
        #     if data_path[-7:] == 'VOC2012':
        #         # this is expected, most pascal voc distibutions dont have the test.txt file
        #         pass
        #     else:
        #         print(e)

        # annotation 파일 read
        annots = [os.path.join(annot_path, s) for s in os.listdir(annot_path)]
        idx = 0

        annots = tqdm(annots)
        for annot in annots:
            # try:
            exist_flag = False
            idx += 1
            annots.set_description("Processing %s" % annot.split(os.sep)[-1])

            et = ET.parse(annot)
            element = et.getroot()

            element_objs = element.findall('object')
            # element_filename = element.find('filename').text + '.jpg'
            element_filename = element.find('filename').text
            element_width = int(element.find('size').find('width').text)
            element_height = int(element.find('size').find('height').text)

            if len(element_objs) > 0:
                annotation_data = {'filepath': os.path.join(imgs_path, element_filename), 'width': element_width,
                                   'height': element_height, 'bboxes': []}

                annotation_data['image_id'] = idx

                if element_filename in trainval_files:
                    annotation_data['imageset'] = 'trainval'
                    exist_flag = True

                if element_filename in train_files:
                    annotation_data['imageset'] = 'train'
                    exist_flag = True

                if element_filename in val_files:
                    annotation_data['imageset'] = 'val'
                    exist_flag = True

                if len(test_files) > 0:
                    if element_filename in test_files:
                        annotation_data['imageset'] = 'test'
                        exist_flag = True

                    # if element_filename in trainval_files:
                    #     annotation_data['imageset'] = 'trainval'
                    # elif element_filename in test_files:
                    #     annotation_data['imageset'] = 'test'
                    # else:
                    #     annotation_data['imageset'] = 'trainval'

            # annotation file not exist in ImageSet
            if not exist_flag:
                continue

            for element_obj in element_objs:
                class_name = element_obj.find('name').text
                if class_name not in classes_count:
                    classes_count[class_name] = 1
                else:
                    classes_count[class_name] += 1

                # class mapping 정보 추가
                if class_name not in class_mapping:
                    class_mapping[class_name] = len(class_mapping)  # 마지막 번호로 추가

                obj_bbox = element_obj.find('bndbox')
                x1 = int(round(float(obj_bbox.find('xmin').text)))
                y1 = int(round(float(obj_bbox.find('ymin').text)))
                x2 = int(round(float(obj_bbox.find('xmax').text)))
                y2 = int(round(float(obj_bbox.find('ymax').text)))
                difficulty = int(element_obj.find('difficult').text) == 1
                annotation_data['bboxes'].append(
                    {'class': class_name, 'x1': x1, 'x2': x2, 'y1': y1, 'y2': y2, 'difficult': difficulty})
            all_imgs.append(annotation_data)

            if visualise:
                img = cv2.imread(annotation_data['filepath'])
                for bbox in annotation_data['bboxes']:
                    cv2.rectangle(img, (bbox['x1'], bbox['y1']), (bbox['x2'], bbox['y2']), (0, 0, 255))
                cv2.imshow('img', img)
                print(annotation_data['imageset'])
                cv2.waitKey(0)

            # except Exception as e:
            #     print(e)
            #     continue
    return all_imgs, classes_count, class_mapping
def save_obj(obj,name):
    with open('obj/'+name+".pkl","wb") as f:
        pickle.dump(obj,f,pickle.HIGHEST_PROTOCOL)
def load_obj(name ):
    with open('obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)
if __name__=="__main__":
    img_data=get_data('/content/gdrive/My Drive')[0]
    save_obj(img_data,"test_1")



Processing 007977.xml:   0%|          | 0/4958 [00:00<?, ?it/s]

Parsing annotation files


Processing 001889.xml: 100%|██████████| 4958/4958 [26:26<00:00,  3.01it/s]


FileNotFoundError: ignored

In [73]:
import numpy as np
import cv2
import random
import copy
def anchor_gen(ratios, scales,resize_image=(224,224),featureMap_size=(14,14),anchor_stride=1):
  '''
  boxes=[x1,y1,x2,y2]
  '''
    rpn_stride=int(resize_image[0]/featureMap_size[0])
    # ratios, scales = np.meshgrid(ratios, scales)
    # ratios, scales = ratios.flatten(), scales.flatten()
    point=[]
    width_heigh=[]
    boxes=[]
    for ix in np.arange(featureMap_size[1]):
      for iy in np.arange(featureMap_size[0]):
        point.append((ix*rpn_stride,iy*rpn_stride))
    for scale in scales:
      for ratio in ratios:
        width = scale/ np.sqrt(ratio)
        height = scale * np.sqrt(ratio)
        width_heigh.append((width,height))
    
    for ix,iy in point:
        for ratio_W_H in width_heigh:
          width,height=ratio_W_H[0],ratio_W_H[1]
          coordinate=np.array([ix- width,iy-height,ix+ width,iy+height])
          boxes.append(coordinate)

    boxes=np.array(boxes)
    return boxes
anchors=anchor_gen(ratios = [0.5, 1, 1.5],scales = [4, 8, 16])

[[ -5.65685425  -2.82842712   5.65685425   2.82842712]
 [ -4.          -4.           4.           4.        ]
 [ -3.26598632  -4.89897949   3.26598632   4.89897949]
 ...
 [185.372583   196.6862915  230.627417   219.3137085 ]
 [192.         192.         224.         224.        ]
 [194.93605471 188.40408206 221.06394529 227.59591794]]


In [0]:
def accord_bbox(img_data,*input_network):
    resized_width,resized_height=input_network
    num_bboxes=len(img_data["bboxes"])
    width=img_data["width"]
    height=img_data["height"]
    gta = np.zeros((num_bboxes, 4))
    for bbox_num, bbox in enumerate(img_data['bboxes']):
        # get the GT box coordinates, and resize to account for image resizing
        gta[bbox_num, 0] = bbox['x1'] * (resized_width / float(width))
        gta[bbox_num, 2] = bbox['x2'] * (resized_width / float(width))
        gta[bbox_num, 1] = bbox['y1'] * (resized_height / float(height))
        gta[bbox_num, 3] = bbox['y2'] * (resized_height / float(height))
    gta_bbox=gta
    return gta_bbox

In [0]:
def compute_iou(box, boxes, area, areas):
  '''
  box:anchor
  boxes: boxes
  '''
    y1 = np.maximum(box[0], boxes[:, 0])
    x1 = np.maximum(box[1], boxes[:, 1])
    y2 = np.minimum(box[2], boxes[:, 2])
    x2 = np.minimum(box[3], boxes[:, 3])
    interSec = np.maximum(y2-y1, 0) * np.maximum(x2-x1, 0)
    union = areas[:] + area - interSec
    iou = interSec / union
    return iou
def compute_overlap(boxes1, boxes2):
  '''
  boxes1:anchors
  boxes2:groundTruth
  '''
    areas1 = (boxes1[:,3] - boxes1[:,1]) * (boxes1[:,2] - boxes1[:,0])
    areas2 = (boxes2[:,3] - boxes2[:,1]) * (boxes2[:,2] - boxes2[:,0])
    overlap = np.zeros((boxes1.shape[0], boxes2.shape[0]))
    for i in range(boxes2.shape[0]):
        box = boxes2[i]
        overlap[:,i] = compute_iou(box, boxes1, areas2[i], areas1)
    return overlap


In [0]:
def build_rpnTarget(boxes, anchors, config):
    rpn_match = np.zeros(anchors.shape[0], dtype=np.int32)
    rpn_bboxes = np.zeros((config.train_rois_num, 4))

    iou = compute_overlap(anchors, boxes)
    maxArg_iou = np.argmax(iou, axis=1)
    max_iou = iou[np.arange(iou.shape[0]), maxArg_iou]
    postive_anchor_idxs = np.where(max_iou > 0.4)[0]
    negative_anchor_idxs = np.where(max_iou < 0.1)[0]

    rpn_match[postive_anchor_idxs] = 1
    rpn_match[negative_anchor_idxs] = -1
    maxIou_anchors = np.argmax(iou, axis=0)
    rpn_match[maxIou_anchors] = 1

    ids = np.where(rpn_match == 1)[0]
    extral = len(ids) - config.train_rois_num // 2
    if extral > 0:
        ids_ = np.random.choice(ids, extral, replace=False)
        rpn_match[ids_] = 0

    ids = np.where(rpn_match == -1)[0]
    extral = len(ids) - (config.train_rois_num - np.where(rpn_match == 1)[0].shape[0])
    if extral > 0:
        ids_ = np.random.choice(ids, extral, replace=False)
        rpn_match[ids_] = 0

    idxs = np.where(rpn_match == 1)[0]
    ix = 0
    for i, a in zip(idxs, anchors[idxs]):
        gt = boxes[maxArg_iou[i]]

        gt_h = gt[2] - gt[0]
        gt_w = gt[3] - gt[1]
        gt_centy = gt[0] + 0.5 * gt_h
        gt_centx = gt[1] + 0.5 * gt_w

        a_h = a[2] - a[0]
        a_w = a[3] - a[1]
        a_centy = a[0] + 0.5 * a_h
        a_centx = a[1] + 0.5 * a_w

        rpn_bboxes[ix] = [(gt_centy - a_centy) / a_h, (gt_centx - a_centx) / a_w,
                          np.log(gt_h / a_h), np.log(gt_w / a_w)]
        rpn_bboxes[ix] /= config.RPN_BBOX_STD_DEV
        ix += 1
    return rpn_match, rpn_bboxes

In [39]:
anchors=anchor_gen(ratios = [0.5, 1, 2],scales = [4, 8, 16])
print(anchors)

print(anchors.shape)

[[  0   0]
 [  0  16]
 [  0  32]
 ...
 [208 176]
 [208 192]
 [208 208]]
(1764, 2)
